In [1]:
use mavenfuzzyfactory

Commands completed successfully

Total execution time: 00:00:00.006

**Subject: Repeat Visitors**

<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">We've been thinking about customer value based solely on&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">their first session conversion and revenue. But if customers&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">have repeat sessions, they may be more valuable than we&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">thought. If that's the case, we might be able to spend a bit&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">more to acquire them.&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Could you please pull data on how many of our website&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">visitors come back for another session? 2014 to date is good.</span>

In [6]:
select * from website_sessions limit 3

(3 row(s) affected)

Total execution time: 00:00:01.007

website_session_id,created_at,user_id,is_repeat_session,utm_source,utm_campaign,utm_content,device_type,http_referer
1,2012-03-19 08:04:16,1,0,gsearch,nonbrand,g_ad_1,mobile,https://www.gsearch.com
2,2012-03-19 08:16:49,2,0,gsearch,nonbrand,g_ad_1,desktop,https://www.gsearch.com
3,2012-03-19 08:26:55,3,0,gsearch,nonbrand,g_ad_1,desktop,https://www.gsearch.com


In [22]:
select visits - 1 as repeated_session, count(user_id) as users
from
(select user_id, count(is_repeat_session) as visits from website_sessions 
where website_sessions.created_at < '2014-11-01'
AND website_sessions.created_at >= '2014-01-01'
GROUP BY user_id 
having min(is_repeat_session)=0
order by 2 desc) as a
group by visits
order by 1

(4 row(s) affected)

Total execution time: 00:00:01.746

repeated_session,users
0,126813
1,14086
2,315
3,4686


**Subject: Deeper Dive on Repeat**

Ok, so the repeat session data was really interesting to see. <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Now you've got me curious to better understand the behavior&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">of these repeat customers.&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Could you help me understand the minimum, maximum, and&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">average time between the first and second session for&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">customers who do come back? Again, analyzing 2014 to date&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">is probably the right time period.</span>

In [38]:
select avg(diff), min(diff), max(diff)
from (select user_id, count(is_repeat_session) as visits, min(created_at), max(created_at), datediff(max(created_at), min(created_at)) as diff from website_sessions 
where website_sessions.created_at < '2014-11-03'
AND website_sessions.created_at >= '2014-01-01'
GROUP BY user_id 
having min(is_repeat_session) = 0
and count(is_repeat_session) = 2
order by 2 desc ) as a

(1 row(s) affected)

Total execution time: 00:00:01.726

avg(diff),min(diff),max(diff)
33.2437,1,69


**Subject: Repeat Channel Mix**

<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Let's do a bit more digging into our repeat customers.&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Can you help me understand the channels they come back&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">through? Curious if it's all direct type-in, or if we're paying for&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">these customers with paid search ads multiple times.&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Comparing new vs. repeat sessions by channel would be&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">really valuable, if you're able to pull it! 2014 to date is great.</span>